In [1]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
%matplotlib inline

In [10]:
data = pd.read_csv("cces18_common_vv2.csv")
print(data.shape)
data.head()

(35415, 31)


,birthyr,gender,educ,race,marstat,region,National Economy,Income,Trump Approval,Congress Approval,...,M4A,Repeal ACA,Paris Accord,TPP,Ideology,sexuality,inputstate,EPA CO2 Regulate,Iran Deal,Party
0,1950,2,6,2,3.0,1,4.0,3.0,4.0,4.0,...,1.0,2.0,2.0,2.0,2.0,1.0,34,1.0,2.0,1.0
1,1949,2,6,2,6.0,2,3.0,3.0,4.0,4.0,...,1.0,1.0,2.0,2.0,2.0,1.0,26,1.0,2.0,1.0
2,1966,2,3,1,3.0,2,2.0,3.0,1.0,3.0,...,NaN,1.0,1.0,1.0,4.0,1.0,27,2.0,1.0,4.0
3,1970,2,5,1,6.0,2,4.0,3.0,4.0,3.0,...,1.0,2.0,2.0,2.0,4.0,1.0,27,1.0,2.0,1.0
4,1971,2,4,1,3.0,4,1.0,1.0,1.0,2.0,...,2.0,1.0,1.0,1.0,4.0,1.0,8,2.0,1.0,3.0


In [12]:
data_Y = data['Party']
data_X = data.drop(['Party'], axis=1)
print(data_X.shape)
print(data_Y.shape)
data_X.head()

(35415, 30)
(35415,)


,birthyr,gender,educ,race,marstat,region,National Economy,Income,Trump Approval,Congress Approval,...,Wall,M4A,Repeal ACA,Paris Accord,TPP,Ideology,sexuality,inputstate,EPA CO2 Regulate,Iran Deal
0,1950,2,6,2,3.0,1,4.0,3.0,4.0,4.0,...,2.0,1.0,2.0,2.0,2.0,2.0,1.0,34,1.0,2.0
1,1949,2,6,2,6.0,2,3.0,3.0,4.0,4.0,...,2.0,1.0,1.0,2.0,2.0,2.0,1.0,26,1.0,2.0
2,1966,2,3,1,3.0,2,2.0,3.0,1.0,3.0,...,1.0,NaN,1.0,1.0,1.0,4.0,1.0,27,2.0,1.0
3,1970,2,5,1,6.0,2,4.0,3.0,4.0,3.0,...,2.0,1.0,2.0,2.0,2.0,4.0,1.0,27,1.0,2.0
4,1971,2,4,1,3.0,4,1.0,1.0,1.0,2.0,...,1.0,2.0,1.0,1.0,1.0,4.0,1.0,8,2.0,1.0
